# Projeto de SuperComputação - Insper - 2023.2

### Autor: Renato Laffranchi Falcão

Este relatório tem como objetivo apresentar três diferentes soluções para o Problema do Clique Máximo: abordagem exaustiva, abordagem exaustiva paralelizada com OpenMP e abordagem exaustiva distribuída entre nós de um cluster com MPI.

## Instalação e importação de dependências

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import library.program as program

In [ ]:
!mkdir -p src/

## Abordagem Exaustiva

In [ ]:
%%writefile src/recursive_exhaustive.cpp
#include <iostream>
#include <vector>
#include <deque>
#include <fstream>
#include <algorithm>

// Author: Renato Laffranchi Falcão
// 12/2023

// This is a recursive implementation of the exhaustive search algorithm for the maximum clique problem with backtracking optimization.
// The algorithm is based on the following paper:
// "A review on algorithms for maximum clique problems" by Qinghua Wu and Jin-Kao Hao. European Journal of Operational Research. Volume 242, Issue 3, 1 May 2015, Pages 693-709.
// http://dx.doi.org/10.1016/j.ejor.2014.09.064

std::vector<std::vector<int>> readGraph(const std::string& filename, int& numVertices)
// Module to read graph from file, provided as an adjacency matrix.
{
    std::ifstream file(filename);
    int numEdges;
    file >> numVertices >> numEdges;

    std::vector<std::vector<int>> graph(numVertices, std::vector<int>(numVertices, 0));

    for (int i = 0; i < numEdges; i++) {
        int u, v;
        file >> u >> v;
        // The graph is undirected, so we add the edge in both directions
        graph[u - 1][v - 1] = 1;
        graph[v - 1][u - 1] = 1;
    }

    file.close();

    return graph;
}

void clique(std::vector<int>& currentSolution, std::deque<int>& candidateVertices, std::vector<int>& maximumClique, std::vector<std::vector<int>>& graph)
// Recursive function to find the maximum clique
{
    // Update maximum clique found so far
    if ( currentSolution.size() > maximumClique.size() )
    {
        maximumClique = currentSolution;
    }

    // Backtracking optimization: if the size of the current solution plus the amount of
    // candidate vertices left is smaller than the size of the maximum clique found so far,
    // there is no need to continue
    if ((currentSolution.size() + candidateVertices.size()) <= maximumClique.size())
    {
        return;
    }

    // Creates a recursive call for each candidate vertex and its neighbors
    for (int u : candidateVertices)
    {
        candidateVertices.pop_front();
        std::vector<int> largerSolution(currentSolution);
        largerSolution.push_back(u);
        std::deque<int> neighborVertices;
        for (int v : candidateVertices)
        {
            if (graph[u][v] == 1)
            {
                neighborVertices.push_back(v);
            }
        }
        clique(largerSolution, neighborVertices, maximumClique, graph);
    }
    return;
}

std::vector<int> findMaximumClique()
// Main function to manage the execution of the algorithm
{
    // Read graph from file
    int numVertices;
    const std::string filename = "files/graph.txt";
    std::vector<std::vector<int>> graph = readGraph(filename, numVertices);

    // Create support data structures
    std::vector<int> maximumClique;
    std::vector<int> currentSolution;
    std::deque<int> candidateVertices;

    // Fill candidate vertices
    for (int i = 0; i < numVertices; i++)
    {
        candidateVertices.push_back(i);
    }

    // Find maximum clique
    clique(currentSolution, candidateVertices, maximumClique, graph);

    return maximumClique;
}

int main(int argc, char *argv[])
{
    // Find maximum clique
    std::vector<int> maximumClique = findMaximumClique();

    // Optional argument to check the correctness of the answer
    size_t correctAnswerLen = 0;

    if (argc > 1)
    {
        correctAnswerLen = atoi(argv[1]);
    }

    if (correctAnswerLen > 0 && maximumClique.size() != correctAnswerLen)
    {
        std::cout << "Wrong answer: clique size is " << maximumClique.size() << ", correct answer is " << correctAnswerLen << std::endl;
        return 1;
    }

    // Optional argument to print the maximum clique found
    bool verbose = false;

    if (argc > 2)
    {
        std::string arg = argv[2];
        if (arg == "-v" || arg == "--verbose")
        {
            verbose = true;
        }
    }

    if (verbose)
    {
        std::cout << "Maximum clique: ";
        for (int v : maximumClique)
        {
            // Print with corrected vertex index for comparison (starts at 1)
            std::cout << v+1 << " ";
        }
        std::cout << std::endl;
    }

    return 0;
}

## Solução paralelizada

In [ ]:
%%writefile src/recursive_exhaustive_parallel.cpp
#include <iostream>
#include <vector>
#include <deque>
#include <fstream>
#include <algorithm>
#include <omp.h>

// Author: Renato Laffranchi Falcão
// 12/2023

// This is a recursive implementation of the exhaustive search algorithm for the maximum clique problem with backtracking optimization.
// The algorithm is based on the following paper:
// "A review on algorithms for maximum clique problems" by Qinghua Wu and Jin-Kao Hao. European Journal of Operational Research. Volume 242, Issue 3, 1 May 2015, Pages 693-709.
// http://dx.doi.org/10.1016/j.ejor.2014.09.064

std::vector<std::vector<int>> readGraph(const std::string& filename, int& numVertices)
// Module to read graph from file, provided as an adjacency matrix.
{
    std::ifstream file(filename);
    int numEdges;
    file >> numVertices >> numEdges;

    std::vector<std::vector<int>> graph(numVertices, std::vector<int>(numVertices, 0));

    for (int i = 0; i < numEdges; i++) {
        int u, v;
        file >> u >> v;
        // The graph is undirected, so we add the edge in both directions
        graph[u - 1][v - 1] = 1;
        graph[v - 1][u - 1] = 1;
    }

    file.close();

    return graph;
}

void clique(std::vector<int>& currentSolution, std::deque<int>& candidateVertices, std::vector<int>& maximumClique, std::vector<std::vector<int>>& graph)
// Recursive function to find the maximum clique
{
    // Update maximum clique found so far
    if ( currentSolution.size() > maximumClique.size() )
    {
        maximumClique = currentSolution;
    }

    // Backtracking optimization: if the size of the current solution plus the amount of
    // candidate vertices left is smaller than the size of the maximum clique found so far,
    // there is no need to continue
    if ((currentSolution.size() + candidateVertices.size()) <= maximumClique.size())
    {
        return;
    }

    // Creates a recursive call for each candidate vertex and its neighbors
    for (int u : candidateVertices)
    {
        candidateVertices.pop_front();
        std::vector<int> largerSolution(currentSolution);
        largerSolution.push_back(u);
        std::deque<int> neighborVertices;
        for (int v : candidateVertices)
        {
            if (graph[u][v] == 1)
            {
                neighborVertices.push_back(v);
            }
        }
        clique(largerSolution, neighborVertices, maximumClique, graph);
    }
    return;
}

std::vector<int> findMaximumClique()
// Main function to manage the execution of the algorithm
{
    // Read graph from file
    int numVertices;
    const std::string filename = "files/graph.txt";
    std::vector<std::vector<int>> graph = readGraph(filename, numVertices);

    // Create support data structures
    std::vector<int> maximumClique;
    std::deque<int> candidateVertices;

    // Fill candidate vertices
    for (int i = 0; i < numVertices; i++)
    {
        candidateVertices.push_back(i);
    }

    // For each candidate vertex, assign a thread
    // to find the maximum clique based on that vertex neighbors.
    #pragma omp parallel for schedule(dynamic)
    for (size_t u = 0; u < candidateVertices.size(); u++)
    {
        std::vector<int> localMaximumClique;
        std::vector<int> largerSolution;
        largerSolution.push_back(u);
        std::deque<int> neighborVertices;
        for (int v : candidateVertices)
        {
            if (graph[u][v] == 1)
            {
                neighborVertices.push_back(v);
            }
        }
        // Find maximum clique based on the neighbors of the candidate vertex
        clique(largerSolution, neighborVertices, localMaximumClique, graph);

        // Merge each thread results into the global maximum clique
        #pragma omp critical
        {
            if (localMaximumClique.size() > maximumClique.size())
            {
                maximumClique = localMaximumClique;
            }
        }
    }

    return maximumClique;
}

int main(int argc, char *argv[])
{
    // Find maximum clique
    std::vector<int> maximumClique = findMaximumClique();

    // Optional argument to check the correctness of the answer
    size_t correctAnswerLen = 0;

    if (argc > 1)
    {
        correctAnswerLen = atoi(argv[1]);
    }

    if (correctAnswerLen > 0 && maximumClique.size() != correctAnswerLen)
    {
        std::cout << "Wrong answer: clique size is " << maximumClique.size() << ", correct answer is " << correctAnswerLen << std::endl;
        return 1;
    }

    // Optional argument to print the maximum clique found
    bool verbose = false;

    if (argc > 2)
    {
        std::string arg = argv[2];
        if (arg == "-v" || arg == "--verbose")
        {
            verbose = true;
        }
    }

    if (verbose)
    {
        std::cout << "Maximum clique: ";
        for (int v : maximumClique)
        {
            // Print with corrected vertex index for comparison (starts at 1)
            std::cout << v+1 << " ";
        }
        std::cout << std::endl;
    }

    return 0;
}

## Solução para computação distribuída

In [ ]:
%%writefile src/recursive_exhaustive_distributed.cpp
#include <iostream>
#include <vector>
#include <deque>
#include <fstream>
#include <algorithm>
#include <mpi.h>

// Author: Renato Laffranchi Falcão
// 12/2023

// This is a recursive implementation of the exhaustive search algorithm for the maximum clique problem with backtracking optimization.
// The algorithm is based on the following paper:
// "A review on algorithms for maximum clique problems" by Qinghua Wu and Jin-Kao Hao. European Journal of Operational Research. Volume 242, Issue 3, 1 May 2015, Pages 693-709.
// http://dx.doi.org/10.1016/j.ejor.2014.09.064

std::vector<std::vector<int>> readGraph(const std::string& filename, int& numVertices)
// Module to read graph from file, provided as an adjacency matrix.
{
    std::ifstream file(filename);
    int numEdges;
    file >> numVertices >> numEdges;

    std::vector<std::vector<int>> graph(numVertices, std::vector<int>(numVertices, 0));

    for (int i = 0; i < numEdges; i++) {
        int u, v;
        file >> u >> v;
        // The graph is undirected, so we add the edge in both directions
        graph[u - 1][v - 1] = 1;
        graph[v - 1][u - 1] = 1;
    }

    file.close();

    return graph;
}

void clique(std::vector<int>& currentSolution, std::deque<int>& candidateVertices, std::vector<int>& maximumClique, std::vector<std::vector<int>>& graph)
// Recursive function to find the maximum clique
{
    // Update maximum clique found so far
    if ( currentSolution.size() > maximumClique.size() )
    {
        maximumClique = currentSolution;
    }

    // Backtracking optimization: if the size of the current solution plus the amount of
    // candidate vertices left is smaller than the size of the maximum clique found so far,
    // there is no need to continue
    if ((currentSolution.size() + candidateVertices.size()) <= maximumClique.size())
    {
        return;
    }

    // Creates a recursive call for each candidate vertex and its neighbors
    for (int u : candidateVertices)
    {
        candidateVertices.pop_front();
        std::vector<int> largerSolution(currentSolution);
        largerSolution.push_back(u);
        std::deque<int> neighborVertices;
        for (int v : candidateVertices)
        {
            if (graph[u][v] == 1)
            {
                neighborVertices.push_back(v);
            }
        }
        clique(largerSolution, neighborVertices, maximumClique, graph);
    }
    return;
}

int main(int argc, char *argv[])
{
    // Initialize MPI
    MPI_Init(&argc, &argv);
    // rank: ID of process, from 0 to size-1, being "size" the total number of processes
    // size: The total number of processes
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Create variables to store graph data
    int numVertices;
    std::vector<std::vector<int>> graph;

    // Create vector to store maximum clique
    std::vector<int> maximumClique;

    if (rank == 0)
    {
        // Read graph from file
        const std::string filename = "files/graph.txt";
        graph = readGraph(filename, numVertices);
    }

    // Broadcast the number of vertices to all processes
    MPI_Bcast(&numVertices, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (rank != 0)
    {
        // Processes resize the graph vector according to the number of vertices
        graph.resize(numVertices, std::vector<int>(numVertices));
    }

    // Broadcast the graph data to all processes
    for (int i = 0; i < numVertices; ++i) {
        MPI_Bcast(graph[i].data(), numVertices, MPI_INT, 0, MPI_COMM_WORLD);
    }

    // Main process (rank 0), assigns tasks to other processes
    if (rank == 0 && size > 1)
    {
        // Create deque of candidate vertices
        std::deque<int> candidateVertices;

        // Fill candidate vertices
        for (int i = 0; i < numVertices; i++)
        {
            candidateVertices.push_back(i);
        }

        // For each candidate vertex, commision a process
        // to find the maximum clique based on that vertex neighbors.
        int process = 1;
        for (size_t u = 0; u < candidateVertices.size(); u++)
        {
            // Create support vectors
            std::vector<int> currentSolution;
            std::vector<int> neighborVertices;

            // Add current vertex to solution
            currentSolution.push_back(u);

            // Add neighbors of current vertex to neighborVertices
            for (int v : candidateVertices)
            {
                if (graph[u][v] == 1)
                {
                    neighborVertices.push_back(v);
                }
            }

            // Serialize the size of vectors along with the data
            int currentSolutionSize = currentSolution.size();
            int neighborVerticesSize = neighborVertices.size();

            // Send size of currentSolution
            MPI_Send(&currentSolutionSize, 1, MPI_INT, process, process*10, MPI_COMM_WORLD);

            // Send currentSolution data
            MPI_Send(currentSolution.data(), currentSolutionSize, MPI_INT, process, process*11, MPI_COMM_WORLD);

            // Send size of neighborVertices
            MPI_Send(&neighborVerticesSize, 1, MPI_INT, process, process*12, MPI_COMM_WORLD);

            // Send neighborVertices data
            MPI_Send(neighborVertices.data(), neighborVerticesSize, MPI_INT, process, process*13, MPI_COMM_WORLD);

            // Move to the next process using Round-robin assignment
            process = (process % (size - 1)) + 1;
        }
    }

    // Other processes receive the data and execute the clique function
    else if (rank != 0)
    {
        // Receive size of currentSolution
        int currentSolutionSize;
        MPI_Recv(&currentSolutionSize, 1, MPI_INT, 0, rank*10, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

        // Receive currentSolution data
        std::vector<int> currentSolution(currentSolutionSize);
        MPI_Recv(currentSolution.data(), currentSolutionSize, MPI_INT, 0, rank*11, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

        // Receive size of neighborVertices
        int neighborVerticesSize;
        MPI_Recv(&neighborVerticesSize, 1, MPI_INT, 0, rank*12, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

        // Receive neighborVertices data
        std::vector<int> neighborVerticesVector(neighborVerticesSize);
        MPI_Recv(neighborVerticesVector.data(), neighborVerticesSize, MPI_INT, 0, rank*13, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

        // Casts neighborVertices to deque
        std::deque<int> neighborVertices(neighborVerticesVector.begin(), neighborVerticesVector.end());

        // Call clique function
        clique(currentSolution, neighborVertices, maximumClique, graph);

        // Send size of maximum clique found by process
        int maximumCliqueSize = maximumClique.size();
        MPI_Send(&maximumCliqueSize, 1, MPI_INT, 0, rank*14, MPI_COMM_WORLD);

        // Send maximumClique data
        MPI_Send(maximumClique.data(), maximumCliqueSize, MPI_INT, 0, rank*15, MPI_COMM_WORLD);
    }

    // Main process (rank 0) receives the data from other processes and finds the maximum clique
    if (rank == 0 && size > 1)
    {
        for (int process = 1; process < size; process++) {
            // Receive the size of the clique
            int receivedSize;
            MPI_Recv(&receivedSize, 1, MPI_INT, process, process*14, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            // Receive the actual clique data
            std::vector<int> receivedClique(receivedSize);
            MPI_Recv(receivedClique.data(), receivedSize, MPI_INT, process, process*15, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            // Update the overall maximum clique
            if (receivedSize > (int) maximumClique.size())
            {
                maximumClique = receivedClique;
            }
        }
    }

    if (rank == 0)
    {
        // In the event that there is only one process,
        // the main process will call the clique function.
        if (size == 1)
        {
            // Create support vector
            std::vector<int> currentSolution;

            // Create deque of candidate vertices
            std::deque<int> candidateVertices;

            // Fill candidate vertices
            for (int i = 0; i < numVertices; i++)
            {
                candidateVertices.push_back(i);
            }

            // Find maximum clique
            clique(currentSolution, candidateVertices, maximumClique, graph);
        }

        // Optional argument to check the correctness of the answer
        size_t correctAnswerLen = 0;

        if (argc > 1)
        {
            correctAnswerLen = atoi(argv[1]);
        }

        if (correctAnswerLen > 0 && maximumClique.size() != correctAnswerLen)
        {
            std::cout << "Wrong answer: clique size is " << maximumClique.size() << ", correct answer is " << correctAnswerLen << std::endl;
            return 1;
        }

        // Optional argument to print the maximum clique found
        bool verbose = false;

        if (argc > 2)
        {
            std::string arg = argv[2];
            if (arg == "-v" || arg == "--verbose")
            {
                verbose = true;
            }
        }

        if (verbose)
        {
            std::cout << "Maximum clique: ";
            for (int v : maximumClique)
            {
                // Print with corrected vertex index for comparison (starts at 1)
                std::cout << v+1 << " ";
            }
            std::cout << std::endl;
        }
    }

    MPI_Finalize();
    return 0;
}

## Compilação dos arquivos

A compilação dos arquivos em c++ pode ser feita através dos seguintes comandos:

In [ ]:
!g++ -Wall -Ofast src/recursive_exhaustive.cpp -o src/recursive_exhaustive
!g++ -Wall -fopenmp -Ofast src/recursive_exhaustive_parallel.cpp -o src/recursive_exhaustive_parallel
!mpic++ src/recursive_exhaustive_distributed.cpp -o src/recursive_exhaustive_distributed

Para executar as implementações através do cluster montado em sala de aula, os seguinte arquivos de configuração de batch para Slurm foram utilizados, de forma a maximizar o uso do poder computacional disponível:

In [ ]:
%%writefile src/recursive_exhaustive.slurm
#!/bin/bash
#SBATCH --job-name=recursive_exhaustive
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --partition=normal
#SBATCH --mem=3G
#SBATCH --output="outputs/recursive_exhaustive_%j.out"

MAX_CLIQUE_SIZE=$1

echo "Job: $SLURM_JOB_ID"
./recursive_exhaustive "$MAX_CLIQUE_SIZE" --verbose

In [ ]:
%%writefile src/recursive_exhaustive_parallel.slurm
#!/bin/bash
#SBATCH --job-name=recursive_exhaustive_parallel
#SBATCH --nodes=2
#SBATCH --ntasks=3
#SBATCH --partition=normal
#SBATCH --mem=3G
#SBATCH --output="outputs/recursive_exhaustive_parallel_%j.out"

MAX_CLIQUE_SIZE=$1

echo "Job: $SLURM_JOB_ID"
./recursive_exhaustive_parallel "$MAX_CLIQUE_SIZE" --verbose

In [ ]:
%%writefile src/recursive_exhaustive_distributed.slurm
#!/bin/bash
#SBATCH --job-name=recursive_exhaustive_distributed
#SBATCH --nodes=2
#SBATCH --ntasks=3
#SBATCH --partition=normal
#SBATCH --mem=3G
#SBATCH --output="outputs/recursive_exhaustive_distributed_%j.out"

MAX_CLIQUE_SIZE=$1

echo "Job: $SLURM_JOB_ID"
mpirun ./recursive_exhaustive_distributed "$MAX_CLIQUE_SIZE" --verbose

## Comparação dos resultados

In [ ]:
!rm -rf files/
!mkdir files/
!rm -rf outputs/
!mkdir -p outputs/
!rm -rf results/
!mkdir -p results/

In [ ]:
results_path = "results/"
results_absolute_path = os.path.abspath(results_path)
src_path = "src/"
src_absolute_path = os.path.abspath(src_path)

In [ ]:
MIN_NUM_VERTICES = 10
MAX_NUM_VERTICES = 200
STEP = 10

algorithms  = ["recursive_exhaustive", "recursive_exhaustive_parallel", "recursive_exhaustive_distributed"]

program.run_and_save(algorithms, MIN_NUM_VERTICES, MAX_NUM_VERTICES, STEP, where_to_run = "local", results_folder_path = results_absolute_path, src_folder_path = src_absolute_path)
program.plot_results(algorithms, results_folder_path = results_absolute_path)

## Análise de soluções para o Problema do Clique Máximo

Os códigos apresentados implementam uma busca exaustiva recursiva para resolver o problema do clique máximo em grafos, utilizando OpenMP para paralelização e MPI para distribuição em nós de computação. A abordagem algorítmica, baseada no trabalho de Wu e Hao (2015), é robusta e promissora.

### Comparação entre implementações para diferentes tamanhos de grafos

O gráfico abaixo demonstra o tempo de execução para diferentes tamanhos de grafos, para cada uma das três versões implementadas.

![resultado](results/results.png)

Geralmente, espera-se que a execução paralela seja mais rápida que a sequencial. No entanto, neste caso, enquanto o algoritmo sequencial `recursive_exhaustive` e o distribuído `recursive_exhaustive_distributed` escalam conforme esperado com o aumento do número de vértices, a implementação paralela `recursive_exhaustive_parallel` apresenta uma performance inconsistente e inferior ou similar à solução sequencial para todos os tamanhos de grafo. Essa contradição com as expectativas teóricas de melhoria na performance com a paralelização pode ser atribuída principalmente à quantidade de memória necessária, o que é um problema inerente aos algoritmos recursivos. No entanto, por se tratar de diversas threads executanto o mesmo algoritmo, o alto consumo de memória pelas threads independentes pode estar causando uma competição pelo espaço de memória disponível, ocasionando no atraso observado.

### Conclusão

Os resultados obtidos destacam a complexidade de paralelizar algoritmos de busca exaustiva em grafos. Embora a teoria sugira um desempenho superior com paralelização, na prática, a eficácia depende significativamente da natureza do problema e da implementação específica do algoritmo. Com base nas evidências gráficas e na análise dos resultados, sugere-se que futuras otimizações concentrem-se no refinamento do escalonamento de tarefas e na gestão de estado compartilhado. Estratégias como a decomposição de domínio e técnicas de paralelismo mais granulares, que limitam a competição por recursos compartilhados, podem ser exploradas para aprimorar a paralelização. Adicionalmente, a implementação de algoritmos heurísticos, que oferecem aproximações rápidas em detrimento da precisão absoluta, pode ser uma via alternativa para lidar com as limitações observadas.